In [ ]:
#Ver2.와의 차이점 

#그레디언트 부스팅 트리 사용

# Step 3: Prepare Data for Consumption

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [37]:
t_train = pd.read_csv('../data/Titanic/train.csv')
t_test = pd.read_csv('../data/Titanic/test.csv')

In [38]:
#두 파일을 묶어 데이터 전처리 준비
data_cleaner = [t_train, t_test]

In [39]:
t_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [40]:
t_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
#결측치 제거
for dataset in data_cleaner:    
    #결측치를 중간값으로 대체
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

    #결측치를 가장 많은 빈도의 항구로 대체
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #결측치를 중간값으로 대체
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
    #결측치를 0으로 대체
    dataset['Cabin'].fillna(0, inplace = True)

In [42]:
#상호작용/다항식 추가
for dataset in data_cleaner :    
    #가족 수 컬럼 생성 : 부모님 + 형제 자매 + 자신
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
    #객실 여부 컬럼 생성 : 객실이 있으면 1, 없으면 0
    dataset['Have_Cabin'] = 1
    dataset['Have_Cabin'].loc[dataset['Cabin'] == 0 ] = 0
    
    #혼자 여행 여부 추가 : 혼자면 1, 가족이 있으면 0
    dataset['IsAlone'] = 1 
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0  

    #타이틀만 가져와서 칼럼 생성
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    
    #나이별 카테고리 칼럼 생성
    dataset['age_cat'] = pd.cut(dataset['Age'], 
                             bins=[0, 3, 7, 15, 30, 60, 100], 
                             include_lowest=True, 
                             labels=['baby', 'children', 'teenage', 
                                     'young', 'adult', 'old']) 
   
    #이름 길이 컬럼 추가
    for i in data_cleaner :
        i['Name_Len'] = i['Name'].apply(lambda x: len(x))
    
    #티겟값 길이 추가
    for i in data_cleaner :
        i['Fare_Str'] = i['Fare'].apply(lambda x: str(x))
        i['Fare_Len'] = i['Fare_Str'].apply(lambda x: len(x))

C:\Users\a\Anaconda3\lib\site-packages\pandas\core\indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
#타이틀 정리하기
male = ['Capt', 'Col', 'Don',  'Dr', 'Jonkheer',  'Major', 'Master', 'Mr', 'Rev', 'Sir']
for dataset in data_cleaner :
    for i in range(0, len(dataset)) :
        if dataset['Title'][i] in male:
            dataset['Title'][i]  = 'Mr'
        else : 
            dataset['Title'][i] = 'Mrs'

C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
t_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Have_Cabin,IsAlone,Title,age_cat,Name_Len,Fare_Str,Fare_Len
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S,2,0,0,Mr,young,23,7.25,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,1,0,Mrs,adult,51,71.2833,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S,1,0,1,Mrs,young,22,7.925,5
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,1,0,Mrs,adult,44,53.1,4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S,1,0,1,Mr,adult,24,8.05,4
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,0,Q,1,0,1,Mr,young,16,8.4583,6
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,1,1,1,Mr,adult,23,51.8625,7
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,0,S,5,0,0,Mr,baby,30,21.075,6
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,0,S,3,0,0,Mrs,young,49,11.1333,7
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,0,C,2,0,0,Mrs,teenage,35,30.0708,7


In [45]:
#타겟값 설정
t_train.target = t_train['Survived']

C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [46]:
#필요없는 컬럼 제거
drop_column = ['PassengerId','Cabin', 'Ticket', 'Name', 'Fare_Str']
for dataset in data_cleaner : 
    dataset.drop(drop_column, axis=1, inplace = True)

In [47]:
t_train.data = t_train.drop("Survived", axis = 1)

C:\Users\a\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [48]:
t_train.data.shape

(891, 14)

In [49]:
data_dummies = pd.get_dummies(t_train.data)

In [50]:
data_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
Pclass              891 non-null int64
Age                 891 non-null float64
SibSp               891 non-null int64
Parch               891 non-null int64
Fare                891 non-null float64
FamilySize          891 non-null int64
Have_Cabin          891 non-null int64
IsAlone             891 non-null int64
Name_Len            891 non-null int64
Fare_Len            891 non-null int64
Sex_female          891 non-null uint8
Sex_male            891 non-null uint8
Embarked_C          891 non-null uint8
Embarked_Q          891 non-null uint8
Embarked_S          891 non-null uint8
Title_Mr            891 non-null uint8
Title_Mrs           891 non-null uint8
age_cat_baby        891 non-null uint8
age_cat_children    891 non-null uint8
age_cat_teenage     891 non-null uint8
age_cat_young       891 non-null uint8
age_cat_adult       891 non-null uint8
age_cat_old         891 non-null 

In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data_dummies)

X_scaled = scaler.transform(data_dummies)

In [52]:
X_scaled

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.87827773,
        -0.68224668, -0.15911146],
       [-1.56610693,  0.66386103,  0.43279337, ..., -1.138592  ,
         1.46574551, -0.15911146],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.87827773,
        -0.68224668, -0.15911146],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.87827773,
        -0.68224668, -0.15911146],
       [-1.56610693, -0.25833709, -0.4745452 , ...,  0.87827773,
        -0.68224668, -0.15911146],
       [ 0.82737724,  0.20276197, -0.4745452 , ..., -1.138592  ,
         1.46574551, -0.15911146]])

# Step 5: Model Data

In [53]:
X_scaled.shape

(891, 23)

In [58]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state = 0) 
gbrt.fit(X_scaled, t_train.target)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# Step 6: Validate and Implement

In [55]:
t_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Name_Len,Fare_Len,FamilySize,Have_Cabin,IsAlone,Title,age_cat
0,3,male,34.5,0,0,7.8292,Q,16,6,1,0,1,Mr,adult
1,3,female,47.0,1,0,7.0000,S,32,3,2,0,0,Mrs,adult
2,2,male,62.0,0,0,9.6875,Q,25,6,1,0,1,Mr,old
3,3,male,27.0,0,0,8.6625,S,16,6,1,0,1,Mr,young
4,3,female,22.0,1,1,12.2875,S,44,7,3,0,0,Mrs,young
5,3,male,14.0,0,0,9.2250,S,26,5,1,0,1,Mr,teenage
6,3,female,30.0,0,0,7.6292,Q,20,6,1,0,1,Mrs,young
7,2,male,26.0,1,1,29.0000,S,28,4,3,0,0,Mr,young
8,3,female,18.0,0,0,7.2292,C,41,6,1,0,1,Mrs,young
9,3,male,21.0,2,0,24.1500,S,23,5,3,0,0,Mr,young


In [56]:
data_dummies2 = pd.get_dummies(t_test)

In [57]:
data_dummies2

,Pclass,Age,SibSp,Parch,Fare,Name_Len,Fare_Len,FamilySize,Have_Cabin,IsAlone,...,Embarked_Q,Embarked_S,Title_Mr,Title_Mrs,age_cat_baby,age_cat_children,age_cat_teenage,age_cat_young,age_cat_adult,age_cat_old
0,3,34.5,0,0,7.8292,16,6,1,0,1,...,1,0,1,0,0,0,0,0,1,0
1,3,47.0,1,0,7.0000,32,3,2,0,0,...,0,1,0,1,0,0,0,0,1,0
2,2,62.0,0,0,9.6875,25,6,1,0,1,...,1,0,1,0,0,0,0,0,0,1
3,3,27.0,0,0,8.6625,16,6,1,0,1,...,0,1,1,0,0,0,0,1,0,0
4,3,22.0,1,1,12.2875,44,7,3,0,0,...,0,1,0,1,0,0,0,1,0,0
5,3,14.0,0,0,9.2250,26,5,1,0,1,...,0,1,1,0,0,0,1,0,0,0
6,3,30.0,0,0,7.6292,20,6,1,0,1,...,1,0,0,1,0,0,0,1,0,0
7,2,26.0,1,1,29.0000,28,4,3,0,0,...,0,1,1,0,0,0,0,1,0,0
8,3,18.0,0,0,7.2292,41,6,1,0,1,...,0,0,0,1,0,0,0,1,0,0
9,3,21.0,2,0,24.1500,23,5,3,0,0,...,0,1,1,0,0,0,0,1,0,0


In [59]:
data_dummies2.shape

(418, 23)

In [60]:
Y_scaled = scaler.transform(data_dummies2)

In [61]:
predictions = gbrt.predict(Y_scaled)

In [62]:
predictions

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [63]:
predictions = pd.DataFrame(y_pred_class, columns=['Survived'])

In [64]:
test = pd.read_csv('../data/Titanic/test.csv')
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv('y_test_ver_3.csv', sep=",", index = False)

In [ ]:
#score : 0.63157